In [ ]:
import pandas as pd

df = pd.read_csv(r'/content/Reviews Data')


In [ ]:
df

,Unnamed: 0,Date,Username,Content,Rating,Helped,Language
0,843,2024-07-19,periaswamy chandiran,Good software,5.0,0,en
1,838,2024-07-18,SURESH,Nice 👍,4.0,0,fr
2,839,2024-07-18,Ramesh Vimala,Jai Hind ⚔️🇮🇳 IND You development R&D Thank you 🙏,5.0,0,en
3,97,2024-07-18,Shiva Kalyani,"Super app , very very helpful 👍😊😊😊😊",5.0,0,en
4,849,2024-07-17,Padmaprakash .S,Ok,5.0,0,it
...,...,...,...,...,...,...,...
1213,657,2023-05-05,Mr. Hardstone,Why isn't Urdu language in Bhashini? Please in...,3.0,2,en
1214,144,2023-04-08,Tirtha Pratim Das,Wonderful initiative. Suggesting to include a ...,4.0,7,en
1215,582,2023-03-25,Bharat Gupta,Fantastic app for communication across India,5.0,4,en
1216,193,2023-03-24,Sourabh Deoghare,Congratulations for the launch! Suggestion: An...,5.0,4,en


In [ ]:
del df['Unnamed: 0']

In [ ]:
df

,Date,Username,Content,Rating,Helped,Language
0,2024-07-19,periaswamy chandiran,Good software,5.0,0,en
1,2024-07-18,SURESH,Nice 👍,4.0,0,fr
2,2024-07-18,Ramesh Vimala,Jai Hind ⚔️🇮🇳 IND You development R&D Thank you 🙏,5.0,0,en
3,2024-07-18,Shiva Kalyani,"Super app , very very helpful 👍😊😊😊😊",5.0,0,en
4,2024-07-17,Padmaprakash .S,Ok,5.0,0,it
...,...,...,...,...,...,...
1213,2023-05-05,Mr. Hardstone,Why isn't Urdu language in Bhashini? Please in...,3.0,2,en
1214,2023-04-08,Tirtha Pratim Das,Wonderful initiative. Suggesting to include a ...,4.0,7,en
1215,2023-03-25,Bharat Gupta,Fantastic app for communication across India,5.0,4,en
1216,2023-03-24,Sourabh Deoghare,Congratulations for the launch! Suggestion: An...,5.0,4,en


In [ ]:
# Positive : 0
# Negative: 1
# Neutral: 2

print("Number of rows per star rating:")
print(df['Rating'].value_counts())

def map_sentiment(stars_received):
    if stars_received <= 2:
        return 0
    elif stars_received == 3:
        return 1
    else:
        return 2

df['Sentiment'] = [ map_sentiment(x) for x in df['Rating']]

Number of rows per star rating:
Rating
5.0    740
1.0    196
4.0    149
3.0     81
2.0     52
Name: count, dtype: int64


In [ ]:
df.head()

,Date,Username,Content,Rating,Helped,Language,Sentiment
0,2024-07-19,periaswamy chandiran,Good software,5.0,0,en,2
1,2024-07-18,SURESH,Nice 👍,4.0,0,fr,2
2,2024-07-18,Ramesh Vimala,Jai Hind ⚔️🇮🇳 IND You development R&D Thank you 🙏,5.0,0,en,2
3,2024-07-18,Shiva Kalyani,"Super app , very very helpful 👍😊😊😊😊",5.0,0,en,2
4,2024-07-17,Padmaprakash .S,Ok,5.0,0,it,2


In [ ]:
!pip install transformers torch

In [ ]:
!pip install datasets

# Convert dataframe to Hugging Face dataset
from datasets import Dataset

dataset = Dataset.from_pandas(df)
dataset = dataset.remove_columns(["Date", "Username", "Rating", "Helped", "Language"])
dataset = dataset.rename_column("Sentiment", "labels")
dataset = dataset.train_test_split(test_size=0.2)

train_dataset = dataset['train']
test_dataset = dataset['test']


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments

model_name = "ai4bharat/indic-bert"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)

def preprocess_function(examples):
    return tokenizer(examples['Content'], padding=True, max_length=128)

encoded_train_dataset = train_dataset.map(preprocess_function, batched=True)
encoded_test_dataset = test_dataset.map(preprocess_function, batched=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at ai4bharat/indic-bert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/974 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2906: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


Map:   0%|          | 0/244 [00:00<?, ? examples/s]

In [ ]:
#--------------------------------------------------------80% accuracy---------------------------------------------------
"""
!pip install -U accelerate
!pip install -U transformers
from transformers import TrainingArguments, Trainer, DataCollatorWithPadding
import librosa
from sklearn.metrics import confusion_matrix
from sklearn.metrics import confusion_matrix, recall_score, accuracy_score, precision_score, f1_score
import numpy as np
import torch

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    # Ensure labels are on CPU and as numpy arrays
    #labels = labels.cpu().numpy()
    #predictions = predictions.numpy()

    accuracy = accuracy_score(labels, predictions)
    recall = recall_score(labels, predictions, average='weighted')
    precision = precision_score(labels, predictions, average='weighted')
    f1 = f1_score(labels, predictions, average='weighted')

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    fp16=True if torch.cuda.is_available() else False,
    logging_steps=10
)

trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = encoded_train_dataset,
    eval_dataset = encoded_test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer.train()"""


'\n!pip install -U accelerate\n!pip install -U transformers\nfrom transformers import TrainingArguments, Trainer, DataCollatorWithPadding\nimport librosa\nfrom sklearn.metrics import confusion_matrix\nfrom sklearn.metrics import confusion_matrix, recall_score, accuracy_score, precision_score, f1_score\nimport numpy as np\nimport torch\n\ndef compute_metrics(eval_pred):\n    logits, labels = eval_pred\n    predictions = np.argmax(logits, axis=-1)\n\n    # Ensure labels are on CPU and as numpy arrays\n    #labels = labels.cpu().numpy()\n    #predictions = predictions.numpy()\n\n    accuracy = accuracy_score(labels, predictions)\n    recall = recall_score(labels, predictions, average=\'weighted\')\n    precision = precision_score(labels, predictions, average=\'weighted\')\n    f1 = f1_score(labels, predictions, average=\'weighted\')\n\n    return {\n        \'accuracy\': accuracy,\n        \'precision\': precision,\n        \'recall\': recall,\n        \'f1\': f1\n    }\n\ndata_collator =

In [ ]:
"""import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    return metric.compute(predictions=preds, references=p.label_ids)

trainer.evaluate()
"""

'import numpy as np\nfrom datasets import load_metric\n\nmetric = load_metric("accuracy")\n\ndef compute_metrics(p):\n    preds = np.argmax(p.predictions, axis=1)\n    return metric.compute(predictions=preds, references=p.label_ids)\n\ntrainer.evaluate()\n'

In [ ]:
"""model.save_pretrained("fine_tuned_indicbert")
tokenizer.save_pretrained("fine_tuned_indicbert")"""

'model.save_pretrained("fine_tuned_indicbert")\ntokenizer.save_pretrained("fine_tuned_indicbert")'

In [ ]:
#------------------------------------------------84% accuracy------------------------------------------------------------------
"""
!pip install -U accelerate
!pip install -U transformers torch
from transformers import TrainingArguments, Trainer, DataCollatorWithPadding
import librosa
from sklearn.metrics import confusion_matrix
from sklearn.metrics import confusion_matrix, recall_score, accuracy_score, precision_score, f1_score
import numpy as np
import torch

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    # Ensure labels are on CPU and as numpy arrays
    #labels = labels.cpu().numpy()
    #predictions = predictions.numpy()

    accuracy = accuracy_score(labels, predictions)
    recall = recall_score(labels, predictions, average='weighted')
    precision = precision_score(labels, predictions, average='weighted')
    f1 = f1_score(labels, predictions, average='weighted')

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

training_args = TrainingArguments(
    output_dir='./results1',
    evaluation_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    fp16=True if torch.cuda.is_available() else False,
    logging_steps=10
)

trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = encoded_train_dataset,
    eval_dataset = encoded_test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer.train()
"""

'\n!pip install -U accelerate\n!pip install -U transformers torch\nfrom transformers import TrainingArguments, Trainer, DataCollatorWithPadding\nimport librosa\nfrom sklearn.metrics import confusion_matrix\nfrom sklearn.metrics import confusion_matrix, recall_score, accuracy_score, precision_score, f1_score\nimport numpy as np\nimport torch\n\ndef compute_metrics(eval_pred):\n    logits, labels = eval_pred\n    predictions = np.argmax(logits, axis=-1)\n\n    # Ensure labels are on CPU and as numpy arrays\n    #labels = labels.cpu().numpy()\n    #predictions = predictions.numpy()\n\n    accuracy = accuracy_score(labels, predictions)\n    recall = recall_score(labels, predictions, average=\'weighted\')\n    precision = precision_score(labels, predictions, average=\'weighted\')\n    f1 = f1_score(labels, predictions, average=\'weighted\')\n\n    return {\n        \'accuracy\': accuracy,\n        \'precision\': precision,\n        \'recall\': recall,\n        \'f1\': f1\n    }\n\ndata_coll

In [ ]:
#--------------------------------------------------------------------------------------------86% accuracy------------------------------------------------------------------
"""
!pip install -U accelerate
!pip install -U transformers torch
from transformers import TrainingArguments, Trainer, DataCollatorWithPadding
import librosa
from sklearn.metrics import confusion_matrix
from sklearn.metrics import confusion_matrix, recall_score, accuracy_score, precision_score, f1_score
import numpy as np
import torch

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    #labels are on CPU and as numpy arrays
    #labels = labels.cpu().numpy()
    #predictions = predictions.numpy()

    accuracy = accuracy_score(labels, predictions)
    recall = recall_score(labels, predictions, average='weighted')
    precision = precision_score(labels, predictions, average='weighted')
    f1 = f1_score(labels, predictions, average='weighted')

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

training_args = TrainingArguments(
    output_dir='./results1',
    evaluation_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=4,  #smaller batch size
    gradient_accumulation_steps=2,  # Accumulate gradients
    per_device_eval_batch_size=4,
    num_train_epochs=5,
    weight_decay=0.01,
    fp16=True if torch.cuda.is_available() else False,
    logging_steps=10
)

trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = encoded_train_dataset,
    eval_dataset = encoded_test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer.train()

"""

'\n!pip install -U accelerate\n!pip install -U transformers torch\nfrom transformers import TrainingArguments, Trainer, DataCollatorWithPadding\nimport librosa\nfrom sklearn.metrics import confusion_matrix\nfrom sklearn.metrics import confusion_matrix, recall_score, accuracy_score, precision_score, f1_score\nimport numpy as np\nimport torch\n\ndef compute_metrics(eval_pred):\n    logits, labels = eval_pred\n    predictions = np.argmax(logits, axis=-1)\n\n    #labels are on CPU and as numpy arrays\n    #labels = labels.cpu().numpy()\n    #predictions = predictions.numpy()\n\n    accuracy = accuracy_score(labels, predictions)\n    recall = recall_score(labels, predictions, average=\'weighted\')\n    precision = precision_score(labels, predictions, average=\'weighted\')\n    f1 = f1_score(labels, predictions, average=\'weighted\')\n\n    return {\n        \'accuracy\': accuracy,\n        \'precision\': precision,\n        \'recall\': recall,\n        \'f1\': f1\n    }\n\ndata_collator = D

In [ ]:
!pip install -U accelerate
!pip install -U transformers torch
from transformers import TrainingArguments, Trainer, DataCollatorWithPadding
import librosa
from sklearn.metrics import confusion_matrix
from sklearn.metrics import confusion_matrix, recall_score, accuracy_score, precision_score, f1_score
import numpy as np
import torch

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    #labels are on CPU and as numpy arrays
    #labels = labels.cpu().numpy()
    #predictions = predictions.numpy()

    accuracy = accuracy_score(labels, predictions)
    recall = recall_score(labels, predictions, average='weighted')
    precision = precision_score(labels, predictions, average='weighted')
    f1 = f1_score(labels, predictions, average='weighted')

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

training_args = TrainingArguments(
    output_dir='./results1',
    evaluation_strategy="steps",
    #save_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=8,  #smaller batch size
    per_device_eval_batch_size=8,
    num_train_epochs=4,
    weight_decay=0.01,
    fp16=True if torch.cuda.is_available() else False,
    logging_steps=10,
    load_best_model_at_end=True,
    save_total_limit=1,
    gradient_accumulation_steps=2,  # Effective larger batch size
    dataloader_num_workers=4,  # Parallel data loading
)

trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = encoded_train_dataset,
    eval_dataset = encoded_test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

with torch.no_grad(): # Temporarily disable gradient calculations
    trainer.model.albert.encoder.embedding_hidden_mapping_in.weight = torch.nn.Parameter(trainer.model.albert.encoder.embedding_hidden_mapping_in.weight.contiguous())

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
10,1.066900,1.022237,0.762295,0.581094,0.762295,0.659474
20,0.991400,0.917757,0.762295,0.581094,0.762295,0.659474
30,0.868700,0.705672,0.762295,0.581094,0.762295,0.659474
40,0.654000,0.626716,0.762295,0.581094,0.762295,0.659474
50,0.700300,0.621054,0.762295,0.581094,0.762295,0.659474
60,0.733500,0.611649,0.762295,0.581094,0.762295,0.659474
70,0.566600,0.570554,0.762295,0.581094,0.762295,0.659474
80,0.597700,0.524394,0.762295,0.581094,0.762295,0.659474
90,0.539300,0.538717,0.762295,0.581094,0.762295,0.659474
100,0.610400,0.568924,0.762295,0.581094,0.762295,0.659474


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.f

Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
10,1.066900,1.022237,0.762295,0.581094,0.762295,0.659474
20,0.991400,0.917757,0.762295,0.581094,0.762295,0.659474
30,0.868700,0.705672,0.762295,0.581094,0.762295,0.659474
40,0.654000,0.626716,0.762295,0.581094,0.762295,0.659474
50,0.700300,0.621054,0.762295,0.581094,0.762295,0.659474
60,0.733500,0.611649,0.762295,0.581094,0.762295,0.659474
70,0.566600,0.570554,0.762295,0.581094,0.762295,0.659474
80,0.597700,0.524394,0.762295,0.581094,0.762295,0.659474
90,0.539300,0.538717,0.762295,0.581094,0.762295,0.659474
100,0.610400,0.568924,0.762295,0.581094,0.762295,0.659474


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_clas

ValueError: You are trying to save a non contiguous tensor: `albert.encoder.albert_layer_groups.0.albert_layers.0.attention.query.weight` which is not allowed. It either means you are trying to save tensors which are reference of each other in which case it's recommended to save only the full tensors, and reslice at load time, or simply call `.contiguous()` on your tensor to pack it before saving.